# Classifying Sepsis Patients with Reinforcement Learning

By Zachary Barnes & Mundy Reimer

---

# Introduction

We designed a reinforcement learning environment and model to classify patients with sepsis at each hour.

Sepsis is a life-threatening condition that arises when the body's response to infection causes injury to its tissues and organs. It is the most common cause of death for people who have been hospitalized, and results in a $15.4 billion annual cost in the US. Early detection and treatment are essential for prevention and a 1-hour delay in antibiotic treatment can lead to 4% increase in hospital mortality. Given the nature of our data as a multivariate timeseries of patient vital signs, this makes this an ideal classification problem to apply reinforcement learning methods to.

---

We first need to import our necessary packages as well as suppress any version requirement warnings for a cleaner experience.

In [ ]:
# Filter tensorflow version warnings

import os
# https://stackoverflow.com/questions/40426502/is-there-a-way-to-suppress-the-messages-tensorflow-prints/40426709
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

import warnings
# https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

import logging
tf.get_logger().setLevel(logging.ERROR)

In [1]:
# OpenAI-based environment imports
import gym
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy
from stable_baselines.deepq import DQN, MlpPolicy as DQN_MlpPolicy, LnMlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2, A2C

# Our helper files and self-designed environment
from env.SepsisEnv import SepsisEnv
from load_data import load_data
from add_reward import add_reward_df, add_end_episode_df

# Other imports
import pandas as pd
import numpy as np
import re
import altair as alt
from tqdm import tqdm
from bayes_opt import BayesianOptimization

---

## Load Data

![physionet_logo](images/physionet_logo.jpeg)

We used a public data set from the [PhysioNet Computing Challenge which can be downloaded here](https://physionet.org/content/challenge-2019/1.0.0/).  Information provided by the PhysioNet Challenge regarding the structure of the data is given below:

*Note - Running the below section for the first time will take a couple of minutes.  Once your cache folder is created, these cells should run much quicker in the future.*

In [2]:
df = load_data()
df = add_reward_df(df)
df = add_end_episode_df(df)

100%|██████████| 16269/16269 [10:43<00:00, 25.29it/s]


In [3]:
df = df.reset_index()

Data used in the competition is sourced from ICU patients in three separate hospital systems.

The data repository contains one file per subject (ex - training/p00101.psv). Each training data file provides a table with measurements over time. Each column of the table provides a sequence of measurements over time (ex - heart rate over several hours), where the header of the column describes the measurement. Each row of the table provides a collection of measurements at the same time (ex - heart rate and oxygen level).

![physionet_data_table](images/physionet_data_table.png)

There are 40 time-dependent variables HR, O2Sat, Temp ..., HospAdmTime, which are described here. The final column, SepsisLabel, indicates the onset of sepsis according to the Sepsis-3 definition, where 1 indicates sepsis and 0 indicates no sepsis. Entries of NaN (not a number) indicate that there was no recorded measurement of a variable at the time interval.

![timeseries](images/multivariate_timeseries.png)

---

## Define Models

Our Reinforcement Learning environment is using [OpenAI's gym](https://github.com/openai/gym).

For step-by-step instructions for how to set up your environment, see the *Setup* section in our project's [README.md](https://github.com/zs-barnes/RL-Sepsis-Prediction/blob/master/README.md).

To create this environment, we referenced:
* How to create a custom gym environment with RL training code [here](https://towardsdatascience.com/creating-a-custom-openai-gym-environment-for-stock-trading-be532be3910e).
* Creating RL algorithms using the Stable Baselines package [here](https://github.com/hill-a/stable-baselines).

We can briefly frame our reinforcement learning problem as such:
* Environment: SepsisEnv modeled using OpenAI Gym, where we have a sequential multivariate timeseries of patients' vital signs
* Agent: A binary classifier that predicts whether patients have sepsis or not
* States: Each timestep that contains multiple patient vital signs taken at the same time
* Actions: Binary prediction of whether a patient has sepsis (1) or does not (0)
* Rewards: The calculated score between 1 and -2 based on the utility function calculated from true/false positive and true/false negative rates

The algorithm will be evaluated by its performance as a binary classifier using a utility function created by the [PhysioNet Challenge](https://physionet.org/content/challenge-2019/1.0.0/). This utility function rewards classifiers for early predictions of sepsis and penalizes them for late/missed predictions and for predictions of sepsis in non-sepsis patients.

The PhysioNet defines a score U(s,t) for each prediction, i.e., for each patient s and each time interval t (each line in the data file) as such:

![physionet_utility](images/physionet_utility.png)

The following figure shows the utility function for a sepsis patient with t_sepsis = 48 as an example (figure from [PhysioNet Challenge](https://physionet.org/content/challenge-2019/1.0.0/)):

![physionet_utility_plot](images/physionet_utility_plot.png)

Below we will define our models, the training process, and how to iterate over multiple patients.

In [4]:
def train_model(env, model, total_timesteps, iterations):
    '''
    Inputs
        - env : SepsisEnv created from OpenAI framework
        - model : specific model such as PPO2, DQN, etc
        - total_timesteps : total time steps chosen (main bottleneck)
        - iterations : number of iterations to run model 
        
    Output
        - A list of names, rewards, and patients
    '''
    
    # Initialization of model, observations, rewards, and patients
    model.learn(total_timesteps=total_timesteps)
    obs = env.reset()
    reward_list = []
    patient_list = []
    patient_count = 0
    
    # Run training loop
    for _ in tqdm(range(iterations)):
        
        # Predict sepsis or not
        action, _states = model.predict(obs)
        
        # Calculate utility from true/false positive/negative rates
        obs, rewards, done, info = env.step(action)
        reward_list.append(rewards[0])
        
        # Reset and redo above for each patient
        # since each patient has their own corresponding timeseries
        patient_list.append(patient_count)
        if done:
            patient_count += 1           
            obs = env.reset()
        # env.render()
        
    # Print results
    model_name = re.sub(r'\W+', '', str(model.__class__).split('.')[-1])
    policy_name = re.sub(r'\W+', '', str(model.policy).split('.')[-1])
    print('Model: ', model_name)
    print('Policy: ', policy_name)
    print('Total patients: ', patient_count)
    print('Total reward:', sum(reward_list))
    
    # Return model, policy, rewards, patients
    name = model_name + ' ' + policy_name
    return (name, reward_list), patient_list

In [5]:
def train_baseline_models(df, iterations, constant=False):
    '''
    Inputs
        - df : data
        - iterations : number of iterations to train model
        - constant : used to set learning rate
    Output
        - Prints model, patient count, and rewards
        - Returns names and reward list
    '''
    
    # Initialization of rewards, patients, 
    # Sepsis environment, and observations
    reward_list = []
    patient_list = []
    env = DummyVecEnv([lambda: SepsisEnv(df)])
    obs = env.reset()
    patient_count = 0
    
    # Run training loop
    for _ in tqdm(range(iterations)): 
        
        # Either get observations and rewards for a patient
        if constant:
            obs, rewards, done, info = env.step(np.array([0]))  
        # Or predict sepsis and then get observations and rewards
        else:
            action = np.random.choice([0,1], size=1)
            obs, rewards, done, info = env.step(action)
        reward_list.append(rewards[0])
        patient_list.append(patient_count)
        
        # Reset and redo above for each patient
        # since each patient has their own corresponding timeseries
        if done:
            patient_count += 1
            obs = env.reset()
            
    # Print results
    if constant:
        name = 'Constant'
        print(f'Model: {name}')
    else:
        name = 'Random'
        print(f'Model: {name}')
    print('Total patients: ', patient_count)
    print('Total reward:', sum(reward_list))
    
    # Return model and rewards
    return (name, reward_list)

---

## Set up environment and models

We instantiate our environment and list of policies + models that we will be comparing.

In total, we compare:
* Proximal Policy Optimization Algorithm + Multi-Layer Perceptron
* Proximal Policy Optimization Algorithm + Multi-Layer Perceptron Long-Short Term Memory
* Proximal Policy Optimization Algorithm + Multi-Layer Perceptron Long-Short Term Memory with Layer Normalization
* Synchronous, deterministic variant of Asynchronous Advantage Actor Critic + Multi-Layer Perceptron
* Synchronous, deterministic variant of Asynchronous Advantage Actor Critic + Multi-Layer Perceptron Long-Short Term Memory
* Deep Q Network + Multi-Layer Perceptron
* Deep Q Network + Multi-Layer Perceptron Long-Short Term Memory

In [6]:
env = DummyVecEnv([lambda: SepsisEnv(df)])

In [7]:
models = [
    PPO2(MlpPolicy, env, verbose=0),
    PPO2(MlpLstmPolicy, env, nminibatches=1, verbose=0), 
    PPO2(MlpLnLstmPolicy, env, nminibatches=1, verbose=0), 
    A2C(MlpPolicy, env, lr_schedule='constant'), 
    A2C(MlpLstmPolicy, env, lr_schedule='constant'),
    DQN(env=env,
        policy=DQN_MlpPolicy,
        learning_rate=1e-3,
        buffer_size=50000,
        exploration_fraction=0.1,
        exploration_final_eps=0.02,
        ),
    DQN(env=env,
        policy=LnMlpPolicy,
        learning_rate=1e-3,
        buffer_size=50000,
        exploration_fraction=0.1,
        exploration_final_eps=0.02,
        )
]

We can reduce the total timesetps and number of iterations if needed for demo purposes.  For reference, setting total_timesteps=20,000 and iterations=50,000 will take ~20 minutes to run locally on a MacBook Pro (2.6 GHz Intel Core i7, 16 GB) 

In [8]:
total_timesteps = 20_000
iterations = 50_000

---

## Train RL models

Now we will actually train and run our models.  This step will take awhile depending upon the numbers set in the previous cell.

In [9]:
rewards = []
patients = []
for model in models:
    env = DummyVecEnv([lambda: SepsisEnv(df)])
    reward_list = train_model(env=env, model=model, total_timesteps=total_timesteps, iterations=iterations)
    rewards.append(reward_list[0])
    patients.append(reward_list[1])

100%|██████████| 50000/50000 [02:14<00:00, 372.09it/s]


Model:  PPO2
Policy:  MlpPolicy
Total patients:  1316
Total reward: -1138.4833536855876


100%|██████████| 50000/50000 [02:30<00:00, 332.22it/s]


Model:  PPO2
Policy:  MlpLstmPolicy
Total patients:  1316
Total reward: -1152.5777983665466


100%|██████████| 50000/50000 [02:40<00:00, 311.26it/s]


...

Model:  A2C
Policy:  MlpLstmPolicy
Total patients:  1316
Total reward: -1159.3166870437562


100%|██████████| 50000/50000 [02:13<00:00, 374.05it/s]


Model:  DQN
Policy:  MlpPolicy
Total patients:  1316
Total reward: -848.8166822530329


100%|██████████| 50000/50000 [02:10<00:00, 384.54it/s]

Model:  DQN
Policy:  LnMlpPolicy
Total patients:  1316
Total reward: -561.155566483736


---

## Get Baseline Performance

Compare RL models against a constant prediction of 0 and a random model of 0 or 1.

In [10]:
reward_list = train_baseline_models(df=df, iterations=iterations, constant=False)
rewards.append(reward_list)
reward_list = train_baseline_models(df=df, iterations=iterations, constant=True)
rewards.append(reward_list)

  0%|          | 44/50000 [00:00<01:54, 436.90it/s]

Model: Random
Total patients:  1316
Total reward: -1363.8666896298528


100%|██████████| 50000/50000 [01:38<00:00, 508.58it/s]

Model: Constant
Total patients:  1316
Total reward: -1138.8889092504978


---

## Visualization & Interpretation

We gather the total reward over each patient and get a cumulative sum for the visualizations.

In [11]:
reward_df = pd.DataFrame(dict(rewards))
reward_df['patients'] = patients[0]
reward_df = reward_df.groupby('patients').sum()
pivot = pd.melt(reward_df.cumsum().reset_index(),id_vars='patients', value_vars=[c for c in reward_df.columns])
pivot.value = pivot.value.round()

In [12]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

Since we are using a third-party plotting package called [Altair](https://altair-viz.github.io/gallery/multiline_highlight.html), for the purposes of rendering this notebook within the browser on Github, we will save our data in our `pivot.csv` file and also show a previously made plot.  This plot can be generated by the code in the sections below.

In [13]:
pivot = pd.read_csv('pivot.csv')

We then compared performance across multiple algorithms.  The below plot nicely summarizes our results, with both versions of our Deep Q-Learning Network with Multi-Layer Perceptrons performing the best, all the combinations of A2C and Proximal Policy models performing worse than the Deep Q-Learning Networks, and our random baseline model performing the worst as expected:

![visualization_anim](images/visualization_anim.png)

Having both our Deep Q-Learning Networks perform the best makes sense since it combines Q-Learning with the power of deep neural networks to let RL work for complex, high-dimensional environments like our multivariate space of all the patient's vital signs.

For future direction, we will attempt to tease out the differences between the different Deep Q-Learning Networks and explore the potential benefits or pitfalls of providing layer normalization versus not.  Since this is medical data where each feature is interpretable, this also lends itself quite well to feature engineering depending upon domain expertise input provided by a medical professional.  We can also run this on alternative data sets to validate our work outside these three hospitals to see if this is generalizable.

---

## Interactive Charts

We reference this for [how to make nice plots in Altair](https://altair-viz.github.io/gallery/multiline_highlight.html).

In [14]:
highlight = alt.selection(type='single', on='mouseover',
                          fields=['variable'], nearest=True)

alt.Chart(pivot).mark_line().encode(
    x='patients',
    y='value',
    color='variable'
)

base = alt.Chart(pivot).mark_line().encode(
    x=alt.X('patients', axis=alt.Axis(title='Patients Observed')),
    y=alt.Y('value', axis=alt.Axis(title='Cumulative Rewards')),
    color=alt.Color('variable', legend=alt.Legend(title='Model and Policy'))
)
points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)

points + lines


alt.LayerChart(...)

From https://altair-viz.github.io/gallery/multiline_tooltip.html

In [15]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['patients'], empty='none')

# The basic line
line = alt.Chart(pivot).mark_line(interpolate='basis').encode(
    x=alt.X('patients', axis=alt.Axis(title='Patients Observed')),
    y=alt.Y('value', axis=alt.Axis(title='Cumulative Rewards')),
    color=alt.Color('variable', legend=alt.Legend(title='Model and Policy'))
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(pivot).mark_point().encode(
    x='patients:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'value:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(pivot).mark_rule(color='gray').encode(
    x='patients:Q',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)

alt.LayerChart(...)

In [16]:
pivot.to_csv('pivot.csv')

---

## Hyperparameter Tuning

We were also able to tune our hyperparameters using bayesian optimization.

We referenced our Bayesian Optimization code from [here](https://colab.research.google.com/gist/iyaja/bf1d35a09ea5e0559900cc9136f96e36/hyperparameter-optimization-fastai.ipynb#scrollTo=gGZm73Txs9PS). 

In [ ]:
df = load_data()
df = add_reward_df(df)
df = add_end_episode_df(df)

In [ ]:
df = df.reset_index()

In [ ]:
total_timesteps = 10_000
iterations = 50_000

In [ ]:
def train_model(env, model, total_timesteps, iterations):
    '''
    Inputs
        - df : data
        - iterations : number of iterations to train model
        - constant : used to set learning rate
    Output
        - Returns sum of reward list
    '''
    model.learn(total_timesteps=total_timesteps)
    reward_list = []
    obs = env.reset()
    patient_count = 0
    for _ in tqdm(range(iterations)):
        action, _states = model.predict(obs)
        obs, rewards, done, info = env.step(action)
        reward_list.append(rewards[0])
        if done:
            patient_count += 1           
            obs = env.reset()
    model_name = re.sub(r'\W+', '', str(model.__class__).split('.')[-1])
    policy_name = re.sub(r'\W+', '', str(model.policy).split('.')[-1])
    
    return sum(reward_list)

In [ ]:
def fit_with(lr, bs, eps, final_eps):
    '''
    Inputs
        - lr : learning rate
        - bs : buffer size
        - eps : exploration factor epsilon
        - final_eps : final exploration factor epsilon
    Output
        - Total reward
    '''
    env = DummyVecEnv([lambda: SepsisEnv(df)])
    model = DQN(env=env,
    policy=DQN_MlpPolicy,
    learning_rate=lr,
    buffer_size=bs,
    exploration_fraction=eps,
    exploration_final_eps=final_eps,
    )
    total_reward = train_model(env=env, model=model, total_timesteps=total_timesteps, iterations=iterations)
    return total_reward

In [ ]:
# Bounded region of parameter space
pbounds = {'lr': (1e-2, 1e-4), 'bs':(5_000, 100_000), 'eps':(0.01, 0.2), 'final_eps': (0.01, 0.02)}
optimizer = BayesianOptimization(
    f=fit_with,
    pbounds=pbounds,
    verbose=2 
)

optimizer.maximize(init_points=2, n_iter=5,)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print('Max', optimizer.max)